In [ ]:
%load_ext autoreload

import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')

import numpy as np
import numpy.linalg as la
import scipy as sp
import scipy.io as spio

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib as mpl
import seaborn as sns
import matplotlib.colors as mcolors


import itertools as it
import queue
import networkx as nx
import skimage.morphology as morph

import homology

mpl.rcParams['mathtext.fontset'] = 'cm'
sns.set_context('poster', font_scale=1.25)
sns.set(color_codes=True, palette='deep')
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})

In [ ]:
# mat = spio.loadmat("../sample_data/Z17.mat")
# data = mat['Z17'][1500:2000, 1500:2000]

mat = spio.loadmat("../sample_data/thresholded/Creases15.mat")
data = mat['ridges'][1750:2000, 1750:2000].astype(np.int8)

print(data.shape)


# # data = np.array([[1,0,-1,0,1],
# #                 [0,1,0,1,0],
# #                 [-1,0,-1,0,-1],
# #                 [0,1,0,1,0],
# #                 [1,0,-1,0,1]])

# # data = mat['ridges'][1000:2000, 1000:2000].astype(np.int8)+mat['valleys'][1000:2000, 1000:2000].astype(np.int8)

# # data = mat['ridges'].astype(np.int8)

fig, ax = plt.subplots(figsize=(16,16))
im = ax.imshow(data, cmap=plt.cm.Blues_r)
ax.axis('off')
# plt.colorbar(im)
plt.show()

In [ ]:
G, positions = homology.construct_mesh_graph(data.shape[0], data.shape[1], diagonals=True, pos=True)

segments = homology.compute_graph_segmentation(G, -data.flatten(), True, positions)


In [ ]:
palette = it.cycle(sns.color_palette("deep"))

image = np.zeros([data.shape[0]*data.shape[1], 3])

for i in segments:
    if i == 0:
        image[list(segments[i])] = (0.0, 0.0, 0.0)
    else:
        color = next(palette)
        image[list(segments[i])] = color

fig, ax = plt.subplots(figsize=(16, 16))
ax.imshow(image.reshape((data.shape[0], data.shape[1], 3)))
ax.axis('off')
plt.show()

In [ ]:
G, positions = homology.construct_mesh_graph(data.shape[0], data.shape[1], diagonals=True, pos=True)

                                             
dist = homology.compute_graph_dilation(G, np.where(data.flatten() == 1)[0], euclidean=True, positions=positions)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))    
ax.imshow(dist.reshape(data.shape), cmap=plt.cm.Blues_r)
ax.axis('off')
plt.show()

In [ ]:
%autoreload

segments = homology.compute_graph_segmentation(G, -dist)

In [ ]:
palette = it.cycle(sns.color_palette("deep", 6))

image = np.zeros([data.shape[0]*data.shape[1], 3])

for i in segments:
    if i == 0:
        image[list(segments[i])] = (0.0, 0.0, 0.0)
    else:
        color = next(palette)
        image[list(segments[i])] = color

# image[np.where(data.flatten() == 1)[0]] = (0.0, 0.0, 0.0)

# image[np.where(dist > 27 )[0]] = (1.0, 1.0, 1.0)

fig, ax = plt.subplots(figsize=(32,32))
ax.imshow(image.reshape((data.shape[0], data.shape[1], 3)), cmap=plt.cm.gray)
ax.axis('off')
plt.show()

In [ ]:
%autoreload

heights = data.flatten()

print("Creating Mesh")
comp = homology.construct_mesh_complex(data.shape[0], data.shape[1], False)


# print(data.shape)

# print(comp.faces)

# heights = np.append(heights, np.min(heights)-1)

print("Creating Filtration")
(simp_filt, dims) = homology.construct_lower_star_filtration(comp, np.argsort(heights))


# print(simp_filt)
# print(dims)

print("Calculating Persistence")
(ipairs, hsort, persist, sim_to_pindex) = homology.compute_persistence_pairs(simp_filt, dims, sorted(heights), comp)

print("Persistence Pairs:", ipairs)
print("Infinite Persistence:", persist)

In [ ]:
birth = [[] for i in range(3)]
death = [[] for i in range(3)]
mult = [[] for i in range(3)]
for d in range(3):
    for (i, j) in list(ipairs[d].keys()):
        birth[d].append(hsort[i])
        death[d].append(hsort[j])
        mult[d].append(ipairs[d][(i, j)])

pbirth = [[] for i in range(3)]
for d in range(3):
    for i in (persist[d].keys()):
        pbirth[d].append(hsort[i])

# print(birth)
# print(death)

for d in range(3):
    
    fig = plt.figure(figsize=(8,8))
    
    ax1 = fig.add_subplot(1,1,1)

    ax1.scatter(birth[d], death[d], c=mult[d], marker='.', cmap='Blues', edgecolors='k', linewidths=1.0, s=100)
#     ax1.scatter(birth[d], death[d], c='b', marker='.', s=20)
    
    ax1.scatter(pbirth[d], pbirth[d], marker='s', color='r')

#     ax1.plot(np.linspace(0, 125, 100), np.linspace(0, 125, 100), 'k--')
    
    ax1.plot(np.linspace(np.min(heights), np.max(heights), 100), np.linspace(np.min(heights), np.max(heights), 100), 'k--')
    
    ax1.set_title(r"$d={}$".format(d))

#     ax1.set_xscale('symlog', linthreshx=1e-16)
#     ax1.set_yscale('symlog', linthreshy=1e-16)
    
    ax1.set_xlabel(r"birth")
    ax1.set_ylabel(r"death")
        
    plt.tight_layout()

    plt.show()


In [ ]:
for d in range(2):

    fig = plt.figure(figsize=(8,8))
    
    ax1 = fig.add_subplot(1,1,1)

    ax1.scatter(birth[d], np.array(death[d])-np.array(birth[d]), c=mult[d], marker='.', cmap='Blues', edgecolors='k', linewidths=1.0, s=100)
    
    ax1.scatter(pbirth[d], np.zeros_like(pbirth[d]), marker='s', color='r')
        
    ax1.set_title(r"$d={}$".format(d))


    ax1.set_yscale('log')
    
    ax1.set_ylim(np.min(np.array(death[d])-np.array(birth[d]))/2, 
                 np.max(np.array(death[d])-np.array(birth[d]))*2)
    
    ax1.set_xlabel(r"birth")
    ax1.set_ylabel(r"persistence")
        
    plt.tight_layout()

    plt.show()